In [ ]:
Insatallations 

In [46]:
# Required Libraries
!pip install transformers torch pandas nltk tqdm

import pandas as pd
import os
import re
from collections import defaultdict, Counter
from nltk.tokenize import word_tokenize
import nltk

nltk.download("punkt")

[nltk_data] Downloading package punkt to d:\The Hunger Games
[nltk_data]     Saga\Panem\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Emotion Analysis 

In [47]:
# Define the required Characters 
CHARACTER_NAMES = [
    "Katniss_Everdeen", "Peeta_Mellark", "Gale_Hawthorne", "Effie_Trinket",
    "Finnick_Odair", "Johanna_Mason", "President_Coriolanus_Snow", "Alma_Coin"
]

# Exclude emotions: positive, negative, neutral
def load_nrc_lexicon(filepath="NRC-Emotion-Lexicon-Wordlevel-v0.92.txt"):
    nrc = defaultdict(set)
    exclude_emotions = {"positive", "negative", "neutral"}  
    
    with open(filepath, 'r', encoding='utf-8') as file:
        for line in file:
            line = line.strip()
            if not line or line.startswith('#'):
                continue
            parts = re.split(r'\s+', line)  # handles tabs or spaces
            if len(parts) != 3:
                continue
            word, emotion, association = parts
            if association == '1' and emotion not in exclude_emotions:  # Exclude 'positive', 'negative', 'neutral'
                nrc[word].add(emotion)
    return nrc


In [48]:
# Count emotions using lexicon
def count_emotions(text_list, character_name, lexicon):
    emotion_counter = Counter()
    all_words = []

    # Tokenize and filter words
    for text in text_list:
        words = word_tokenize(text.lower())
        words = [w for w in words if w.isalpha()]
        all_words.extend(words)

    for word in all_words:
        if word in lexicon:
            for emotion in lexicon[word]:
                emotion_counter[emotion] += 1

    return emotion_counter

In [49]:
# Display top 3 emotions
def display_top_emotions(emotion_counter, character_name):
    top_emotions = emotion_counter.most_common(3)
    emotion_line = f"{character_name.replace('_', ' ')}: "
    emotion_line += ", ".join([emotion for emotion, _ in top_emotions])
    print(emotion_line)

In [50]:
# Collecting data in a list for later tabular display
emotion_summary = []

In [51]:
# Loop through each character to count emotions
nrc_lexicon = load_nrc_lexicon()

In [52]:
# Creating a loop for each character
for character_name in CHARACTER_NAMES:
    try:
        df = pd.read_csv(f"data/raw/{character_name}.csv")
        paragraphs = df['paragraph'].dropna().tolist()
        emotion_counts = count_emotions(paragraphs, character_name, nrc_lexicon)

        # Get the top 3 emotions
        top_emotions = emotion_counts.most_common(3)

        # Prepare data for table
        character_data = {
            "Character": character_name.replace("_", " "),
            "Top Emotion 1": top_emotions[0][0] if len(top_emotions) > 0 else None,
            "Score 1": top_emotions[0][1] if len(top_emotions) > 0 else None,
            "Top Emotion 2": top_emotions[1][0] if len(top_emotions) > 1 else None,
            "Score 2": top_emotions[1][1] if len(top_emotions) > 1 else None,
            "Top Emotion 3": top_emotions[2][0] if len(top_emotions) > 2 else None,
            "Score 3": top_emotions[2][1] if len(top_emotions) > 2 else None,
        }

        emotion_summary.append(character_data)

    except FileNotFoundError:
        print(f"❌ File not found for {character_name}. Skipping.")
    except Exception as e:
        print(f"⚠️ Error processing {character_name}: {e}")


In [53]:
# Convert summary data into a DataFrame
summary_df = pd.DataFrame(emotion_summary)

from IPython.display import display

display(
    summary_df.style
    .hide(axis='index')  # Hides the index column
)

Character,Top Emotion 1,Score 1,Top Emotion 2,Score 2,Top Emotion 3,Score 3
Katniss Everdeen,trust,363,fear,336,anticipation,293
Peeta Mellark,trust,228,fear,222,anticipation,206
Gale Hawthorne,fear,174,trust,171,anticipation,134
Effie Trinket,trust,99,anticipation,78,fear,64
Finnick Odair,trust,138,anticipation,125,joy,110
Johanna Mason,trust,99,fear,93,anticipation,77
President Coriolanus Snow,fear,301,trust,292,sadness,269
Alma Coin,trust,75,fear,58,anticipation,47
